#MA707 Report - Investigation (spring 2019, DataHeroes)

## Introduction

After doing all the pre-processing to find the best train test datasets and then fitting in to all the estimator pipelines with their default parameters, the scores were compared for all models with all 4 datasets. The two train-test datasets which gave the best scores for most of the estimator pipelines will then be used in this notebook where the models are investigated using Gridsearch with diferent hyperparameter settings and different combinations of estimator pipelines defined in the [Estimator Pipeline notebook](https://bentley.cloud.databricks.com/#notebook/1607450/command/1607451)

## Contents
1. Setup
2. Hyperparameter Tuning
3. Model evaluation and selection
4. Summary

## 1. Setup

In [5]:
%run "./3. Estimator pipelines"

In [6]:
from sklearn.pipeline        import FeatureUnion, Pipeline
from sklearn.linear_model    import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm             import SVR
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.decomposition   import PCA
from spark_sklearn           import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics         import make_scorer, mean_absolute_error, r2_score


##2. Hyperparameter Tuning

##Model 1: Lasso Regression

Use GridSearchCV to get the best hyper parameters for Lasso Regression model.The hyperparameters are:
- `normalize`. True or False.The regressors X will be normalized before regression by subtracting the mean and dividing by the l2-norm or by their standard deviations. 
- `alpha`. It represents the regularization strength; Regularization improves the conditioning of the problem and reduces the variance of the estimates. Here we chose a range (0.001, 1000).

Then cross validation is defined as 5 time series splits which means it will train the model on combination of 4 subsets created from the training datset and validate the trained model on one subset. And the scoring method is R square which is a statistical measure of how close the data are to the fitted regression line. Then fit the gridsearchcv with features and target datasets

In [10]:
from spark_sklearn import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics         import make_scorer, mean_absolute_error, r2_score
lasso_run = \
GridSearchCV(sc,
  estimator=get_lasso_pipeline(),
  param_grid={'lso__normalize':[True,False],
              'lso__alpha'    :[10.0**n for n in range(-3,4)]},
  cv=TimeSeriesSplit(n_splits=5),
  scoring=make_scorer(r2_score),
  return_train_score=False,
  n_jobs=-1 
) 

lasso_run.fit(trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser)

display_pdf(est_grid_results_pdf(lasso_run,
                                 est_tag='lasso'))

mean_test_score,lso__alpha,lso__normalize,rank_test_score,est_tag
-1.0777883353815436,1000.0,true,14,lasso
-0.4962761193752223,0.001,false,13,lasso
-0.4208084521954819,0.001,true,12,lasso
-0.2378441931503919,0.01,false,11,lasso
-0.16679711321838303,0.1,false,10,lasso
-0.05850512669485285,0.01,true,9,lasso
0.0036630498172437525,0.1,true,8,lasso
0.041672421712597545,1.0,false,7,lasso
0.1131349719991033,1.0,true,6,lasso
0.26009369754701733,10.0,true,5,lasso


In [11]:
lasso_run.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser)

display_pdf(est_grid_results_pdf(lasso_run,
                                 est_tag='lasso'))

mean_test_score,lso__alpha,lso__normalize,rank_test_score,est_tag
-1.2553801004928127,0.001,true,14,lasso
-1.0642620033355898,1000.0,true,13,lasso
-0.7723335184166166,0.001,false,12,lasso
-0.6591212599855719,0.01,false,11,lasso
-0.48409118524547323,0.01,true,10,lasso
-0.048067352952655296,0.1,true,9,lasso
0.15615079325770861,0.1,false,8,lasso
0.20858237575100236,1.0,true,7,lasso
0.312894925231664,10.0,true,6,lasso
0.48108440411410164,1000.0,false,5,lasso


The scores for the model with the best hyperparameters for the model trained and validated with the two datasets are:<table>
    <tr>
        <td>feature </td><td>target</td><td>`normalize`</td><td>`alpha`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>false</td><td>100</td><td>54.66%</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>false</td><td>10</td><td>60.67%</td>
    </tr>
    
</table>

- From the results it can be seen that when the hyperparameter `normalize` which normalizes the values between 0.0 and 1.0 is `false`, it consistantly produces the best predictions. 
- The best model for prediction using the lasso regression is when `normalize='False'` and `alpha` is 10 from a range of (0.001, 1000). The R-sqare value using these hyperparameter is 60.67%, that is 60.67% of the variance in the response target variable can be explained using this model.

## Model 2: Lasso with PCA

Use GridSearchCV to get the best hyper parameters for Lasso Regression model with PCA.The hyperparameters tested are:
- `normalize`: `True` or `False`.The regressors X will be normalized before regression by subtracting the mean and dividing by the l2-norm or by their standard deviations. 
- `alpha`: It represents the regularization strength; Regularization improves the conditioning of the problem and reduces the variance of the estimates. Here we chose a range (0.001, 1000).
- `n_components`: It indicates how many principle components are created with the existing features. Principle components explain the variabilities of features.

Then cross validation is done using 5 time series splits and then the `mean_test_scores` for all the cross-validated tests are recorded. Then fit the GridsearchCV with features and target datasets

In [16]:
from spark_sklearn import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics         import make_scorer, mean_absolute_error, r2_score, mean_squared_error
pca_lasso_run = \
GridSearchCV(sc,
  estimator=get_pca_lasso_pipeline(),
  param_grid={'pca__n_components': [10**n for n in [2,3,4]],
              'lso__normalize':[True,False],
              'lso__alpha'    :[10.0**n for n in range(-3,4)]},
  cv=TimeSeriesSplit(n_splits=5),
  scoring=make_scorer(r2_score),
  return_train_score=False,
  n_jobs=-1 
) 

pca_lasso_run.fit(trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser)

display_pdf(est_grid_results_pdf(pca_lasso_run,
                                 est_tag='pca-lasso'))

mean_test_score,lso__alpha,lso__normalize,pca__n_components,rank_test_score,est_tag
-3.08860051069401E20,0.001,true,10000,42,pca-lasso
-2.989747710851224E20,0.01,true,10000,41,pca-lasso
-2.1550461020675912E20,0.1,true,10000,40,pca-lasso
-2.1137778296495368E20,0.001,true,1000,39,pca-lasso
-2.0694435114591496E20,0.01,true,1000,38,pca-lasso
-1.6712486469149465E20,0.1,true,1000,37,pca-lasso
-8.7690110913253212E18,1.0,true,10000,36,pca-lasso
-8.7597421368744212E18,1.0,true,1000,35,pca-lasso
-12.40880657834584,0.001,false,1000,33,pca-lasso
-12.40880657834584,0.001,false,10000,33,pca-lasso


In [17]:
pca_lasso_run.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser)

display_pdf(est_grid_results_pdf(pca_lasso_run,
                                 est_tag='pca-lasso-ironore'))

mean_test_score,lso__alpha,lso__normalize,pca__n_components,rank_test_score,est_tag
-2.3022444525288644E22,0.001,true,10000,42,pca-lasso-ironore
-2.232102219375934E22,0.01,true,10000,41,pca-lasso-ironore
-1.755418163734597E22,0.1,true,10000,40,pca-lasso-ironore
-1.5540514792128878E22,0.001,true,1000,39,pca-lasso-ironore
-1.5253865943142406E22,0.01,true,1000,38,pca-lasso-ironore
-1.2730247708452002E22,0.1,true,1000,37,pca-lasso-ironore
-3.868725755844279E19,1.0,true,10000,36,pca-lasso-ironore
-3.868722572373937E19,1.0,true,1000,35,pca-lasso-ironore
-35.27154067639244,0.001,false,10000,33,pca-lasso-ironore
-35.27154067639244,0.001,false,1000,33,pca-lasso-ironore


From the below test results we can get the best parameters:<table>
    <tr>
        <td>feature </td><td>target</td><td>`normalize`</td><td>`alpha`</td><td>`pca_n_components`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>`true`</td><td>10</td><td>10000</td><td>58.18%</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>`false`</td><td>0.01</td><td>100</td><td>63.43%</td>
    </tr>
    
</table>

- From the GridsearchCV results with the two training datasets, it can be seen that with the `trn_coal_cnt_` datset, when `alpha=10` and normalization is performed `normalize='True'`, it consistently produces better R-square values in the range of 58%. Whereas for the `trn_ore_tfidf_` dataframe the result is consistent with an alpha value in the range of 0.001 to 1. In this case the pca component number is consistent at 100.
- The best mean R-square value is 63.43% for the model trained with the `trn_ore_tfidf_` with hyperparameters `normalize='False'`, `alpha=0.01` and `pca_n_component=100`.

**Note** All the scores are better when compared with the default parameters ran in the estimator pipeline notebook.

##Model 3: ElasticNet

Use GridSearchCV to get the best hyper parameters for the ElasticNet model.The hyperparameters are:
- `normalize`: True or False.The regressors X will be normalized before regression by subtracting the mean and dividing by the l2-norm or by their standard deviations. 
- `alpha`: It represents the regularization strength; Regularization improves the conditioning of the problem and reduces the variance of the estimates. Here we chose a range (0.001, 1000).

Then cross validation is defined as 5 time series splits. Then fit the gridsearchsv with features and target datasets

In [22]:
from spark_sklearn import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics         import make_scorer, mean_absolute_error, r2_score
elasticnet_run = \
GridSearchCV(sc,
  estimator=get_elasticnet_pipeline(),
  param_grid={'ela__normalize':[True,False],
              'ela__alpha'    :[10.0**n for n in range(-3,4)]},
  cv=TimeSeriesSplit(n_splits=5),
  scoring=make_scorer(r2_score),
  return_train_score=False,
  n_jobs=-1 
) 

elasticnet_run.fit(trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser)

display_pdf(est_grid_results_pdf(elasticnet_run,
                                 est_tag='elasticnet'))

mean_test_score,ela__alpha,ela__normalize,rank_test_score,est_tag
-1.0777883353815436,1000.0,true,14,elasticnet
-1.0770883440533305,100.0,true,13,elasticnet
-1.0743315957871369,10.0,true,12,elasticnet
-1.0497357272338996,1.0,true,11,elasticnet
-0.851089572736258,0.1,true,10,elasticnet
-0.6021930304417984,0.01,true,9,elasticnet
-0.5299088546907135,0.001,true,8,elasticnet
0.31501308461484684,0.001,false,7,elasticnet
0.4019259636936047,0.1,false,6,elasticnet
0.45694876637894827,0.01,false,5,elasticnet


In [23]:
elasticnet_run.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser)

display_pdf(est_grid_results_pdf(elasticnet_run,
                                 est_tag='elasticnet-ironore'))

mean_test_score,ela__alpha,ela__normalize,rank_test_score,est_tag
-1.0642620033355898,1000.0,true,14,elasticnet-ironore
-1.063567919954867,100.0,true,13,elasticnet-ironore
-1.0578838955472776,10.0,true,12,elasticnet-ironore
-1.0052938482773788,1.0,true,11,elasticnet-ironore
-0.7613472250418382,0.1,true,10,elasticnet-ironore
-0.5551056728835511,0.01,true,9,elasticnet-ironore
-0.5043259506962956,0.001,true,8,elasticnet-ironore
0.4812450560985701,10.0,false,7,elasticnet-ironore
0.4845148814049471,100.0,false,6,elasticnet-ironore
0.49574596319874975,1000.0,false,5,elasticnet-ironore


From the below test results we can get the best hyperparameters:<table>
    <tr>
        <td>feature </td><td>target</td><td>`normalize`</td><td>`alpha`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>`true`</td><td>1000</td><td>48.52%</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>`false`</td><td>0.01</td><td>60.79%</td>
    </tr>
    
</table>

- From both the GridsearchCV output, it can be seen that the best scores for both the models is consistent when the parameter `normalize` is `false`. It consistently produces the `mean_test_score` of above 50% for `alpha` values in the range of 0.001 to 1 for the model trained on the `_ore_tfidf_` dataset.
- When normalize is false, and alpha is 0.01, it gives the best R square value of 60.79%.

##Model 4: ElasticNet with PCA

Similarly run the GridSearchCV on the `get_pca_elasticnet_pipeline()` which runs the object `pca` which is the class FeatureSelectionPCA() and then fitted into the ElasticNet estimator to get the best hyper parameters for the model.The hyperparameters tested are:
- `normalize`: `True` or `False`.The regressors X will be normalized before regression by subtracting the mean and dividing by the l2-norm or by their standard deviations. 
- `alpha`: It represents the regularization strength; Regularization improves the conditioning of the problem and reduces the variance of the estimates. Here we chose a range (0.001, 1000).
- `n_components`: It indicates how many principle components are created with the existing features. Principle components explain the variabilities of features. Here we define the range as `[10^2, 10^4]`

The model is cross validated using the 5 time series splits and the `mean_test_scores` of R-square values for the cross-validation are compard and ranked accordingly. Then fit the GridsearchCV with the two different features and target datasets

In [28]:
from spark_sklearn import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics         import make_scorer, mean_absolute_error, r2_score
pca_elasticnet_run = \
GridSearchCV(sc,
  estimator=get_pca_elasticnet_pipeline(),
  param_grid={'pca__n_components': [10**n for n in [2,3,4]],
              'ela__normalize':[True,False],
              'ela__alpha'    :[10.0**n for n in range(-3,4)]},
  cv=TimeSeriesSplit(n_splits=5),
  scoring=make_scorer(r2_score),
  return_train_score=False,
  n_jobs=-1 
) 

pca_elasticnet_run.fit(trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser)

display_pdf(est_grid_results_pdf(pca_elasticnet_run,
                                 est_tag='pca-elasticnet'))

mean_test_score,ela__alpha,ela__normalize,pca__n_components,rank_test_score,est_tag
-1.5186464355750714E20,0.001,true,10000,42,pca-elasticnet
-1.0766630998109485E20,0.001,true,1000,41,pca-elasticnet
-1.1382828354847908E19,0.01,true,10000,40,pca-elasticnet
-8.8408318590916116E18,0.01,true,1000,39,pca-elasticnet
-1.47943312891127808E17,0.1,true,10000,38,pca-elasticnet
-1.22682728508276864E17,0.1,true,1000,37,pca-elasticnet
-4.826754159701979E14,1.0,true,10000,36,pca-elasticnet
-4.6529127125126844E14,1.0,true,1000,35,pca-elasticnet
-1.0777883353815436,1000.0,true,10000,32,pca-elasticnet
-1.0777883353815436,1000.0,true,100,32,pca-elasticnet


In [29]:
pca_elasticnet_run.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser)

display_pdf(est_grid_results_pdf(pca_elasticnet_run,
                                 est_tag='pca-elasticnet-ironore'))

mean_test_score,ela__alpha,ela__normalize,pca__n_components,rank_test_score,est_tag
-1.0834713640204767E22,0.001,true,10000,42,pca-elasticnet-ironore
-8.015165226161942E21,0.001,true,1000,41,pca-elasticnet-ironore
-7.478940989582114E20,0.01,true,10000,40,pca-elasticnet-ironore
-5.927231873156944E20,0.01,true,1000,39,pca-elasticnet-ironore
-9.65071263893164E18,0.1,true,10000,38,pca-elasticnet-ironore
-7.790491899349633E18,0.1,true,1000,37,pca-elasticnet-ironore
-1.1815796327912372E16,1.0,true,10000,36,pca-elasticnet-ironore
-1.0111897305649016E16,1.0,true,1000,35,pca-elasticnet-ironore
-1557399.7479057803,10.0,true,10000,33,pca-elasticnet-ironore
-1557399.7479057803,10.0,true,1000,33,pca-elasticnet-ironore


From the below test results we can get the best parameters:<table>
    <tr>
        <td>feature </td><td>target</td><td>`normalize`</td><td>`alpha`</td><td>`pca_n_components`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>`true`</td><td>0.001</td><td>100</td><td>69.18%</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>`true`</td><td>0.01</td><td>100</td><td>63.61%</td>
    </tr>
    
</table>

- From the `mean_test_score` results from the GridsearchCV on the two different datasets, the most consistent hyperparameter with better scores for both model is when `normalize='False'`and when `pca_n_component=100`. Although the best score for the two models is when this hyperparameter `normalize` is `'True'` fand the highest at 69.18%, it consistently gives better scores for different `alpha` values as well as `pca_n_components` when the values are not normalized.
- The best `mean_test_score` for the two models is 69.18% with the `_coal_cnt_` dataset pre-processed with the CountVectorizer method with the hyperparameters `normalize='True'`, `alpha=0.001` and `pca_n_component=100`.

##Model 5: Ridge Regression

Using similar hyperparameter values as for Lasso regression for `normalize` and `alpha`, run the GridsearchCV for the Ridge Regression model with different solvers `['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']`

**Note:** Solver to use in the computational routines:

- `auto` chooses the solver automatically based on the type of data.
- `svd` uses a Singular Value Decomposition of X to compute the Ridge coefficients. More stable for singular matrices than ‘cholesky’.
- `cholesky` uses the standard scipy.linalg.solve function to obtain a closed-form solution.
- `sparse_cg` uses the conjugate gradient solver as found in scipy.sparse.linalg.cg. As an iterative algorithm, this solver is more appropriate than `cholesky` for large-scale data (possibility to set tol and max_iter).
- `lsqr` uses the dedicated regularized least-squares routine scipy.sparse.linalg.lsqr. It is the fastest and uses an iterative procedure.
- `sag` uses a Stochastic Average Gradient descent, and ‘saga’ uses its improved, unbiased version named SAGA. Both methods also use an iterative procedure, and are often faster than other solvers when both n_samples and n_features are large. Note that ‘sag’ and ‘saga’ fast convergence is only guaranteed on features with approximately the same scale. You can preprocess the data with a scaler from sklearn.preprocessing.%

In [35]:
from spark_sklearn import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics         import make_scorer, mean_absolute_error, r2_score
ridge_run = \
GridSearchCV(sc,
  estimator=get_ridge_pipeline(),
  param_grid={'rdg__normalize':[True,False],
              'rdg__alpha'    :[10.0**n for n in range(-3,4)],
              'rdg__solver'   :['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']},
  cv=TimeSeriesSplit(n_splits=5),
  scoring=make_scorer(r2_score),
  return_train_score=False,
  n_jobs=-1 
) 

ridge_run.fit(trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser)

display_pdf(est_grid_results_pdf(ridge_run,
                                 est_tag='ridge'))

mean_test_score,rdg__alpha,rdg__normalize,rdg__solver,rank_test_score,est_tag
-3523.155148363567,0.001,false,sparse_cg,98,ridge
-8.401905020588437,0.001,false,auto,96,ridge
-8.401905020588437,0.001,false,cholesky,96,ridge
-8.400445816279792,0.001,false,svd,95,ridge
-4.8567588494641205,0.01,false,sparse_cg,94,ridge
-3.3497559403161183,0.01,false,svd,93,ridge
-3.3497026172059874,0.01,false,auto,91,ridge
-3.3497026172059874,0.01,false,cholesky,91,ridge
-1.06739877106617,1000.0,true,saga,90,ridge
-1.067382785657825,1000.0,true,sag,89,ridge


In [36]:
ridge_run.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser)

display_pdf(est_grid_results_pdf(ridge_run,
                                 est_tag='ridge-ironore'))

mean_test_score,rdg__alpha,rdg__normalize,rdg__solver,rank_test_score,est_tag
-6.905446206860463,0.001,false,svd,98,ridge-ironore
-6.905389974323474,0.001,false,auto,96,ridge-ironore
-6.905389974323474,0.001,false,cholesky,96,ridge-ironore
-6.9028074996181,0.001,false,sparse_cg,95,ridge-ironore
-1.0390349845564508,1000.0,true,lsqr,94,ridge-ironore
-1.0390069530278436,1000.0,true,sag,93,ridge-ironore
-1.0390043159178228,1000.0,true,svd,92,ridge-ironore
-1.0390043159178222,1000.0,true,cholesky,90,ridge-ironore
-1.0390043159178222,1000.0,true,auto,90,ridge-ironore
-1.0390024647871194,1000.0,true,saga,89,ridge-ironore


The top `mean_test_score` values for the two models tested with the hyperparameters as defined above are as below:<table>
    <tr>
        <td>feature </td><td>target</td><td>`normalize`</td><td>`alpha`</td><td>`solver`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>`False`</td><td>1</td><td>`saga`</td><td>51.02%</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>`False`</td><td>1</td><td>`sparse_cg`</td><td>60.24%</td>
    </tr>
    
</table>

Comparing the test scores, it can be observed that when the hyperparameter `normalize` is `'False'` and `alpha=1`, it consistently produces the best predictions for differnt solvers. 
The best score from the gridsearchCV is 60.24% with the hyperparameters `alpha=1`, `normalize='False'` and the `sparse_cg` solver is used in the model.

##Model 6: Ridge Regression with PCA

The same Ridge Regression model is then re-run with the same hyperparameters but with an additional feature reduction method for reducing the dimensionality of data. The `FeatureSelectionPCA` method is run in the pipeline which has a base class `PCA` and the hyperparameter `pca_n_component` is set as [10, 20,30].

So the model runs the Ridge Regression on the dataset with number of features reduced by the `FeatureSelectionPCA` method to 10, 20 and 30 using the Principal Component Analysis. This model is then cross-validated using a 5 time-series split of the training dataset and the R-square values are averaged out for the cross-validation and then are compared for all other hyperparameters.

In [40]:
pca_ridge_run = \
GridSearchCV(sc,
            estimator=Pipeline(steps=[('pca',FeatureSelectionPCA()),
                                      ('rdg',EstimatorRidge())
                                     ]),
            param_grid={'rdg__normalize'   :[True, False],
                        'rdg__alpha'       :[10.0**n for n in [-3,0,3]],
                        'rdg__solver'      :['saga'],
                        'pca__n_components':[10*n for n in [1, 2, 3]]
            },
 cv=TimeSeriesSplit(n_splits=5),
 scoring=make_scorer(r2_score),
 return_train_score=False,
 n_jobs=-1
)
pca_ridge_run \
 .fit(trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser)
display_pdf(est_grid_results_pdf(pca_ridge_run,
                                est_tag='pca-ridge'))

mean_test_score,pca__n_components,rdg__alpha,rdg__normalize,rdg__solver,rank_test_score,est_tag
-1.0731107402291904,10,1000.0,true,saga,18,pca-ridge
-1.073022540431678,20,1000.0,true,saga,17,pca-ridge
-1.0729928128574482,30,1000.0,true,saga,16,pca-ridge
0.4568803409008533,30,0.001,true,saga,15,pca-ridge
0.4752231473205381,10,0.001,true,saga,14,pca-ridge
0.47754185126732945,20,0.001,true,saga,13,pca-ridge
0.4795923137868395,10,1.0,true,saga,12,pca-ridge
0.5063048179131614,30,1.0,true,saga,11,pca-ridge
0.5063355505466489,20,1.0,true,saga,10,pca-ridge
0.5066571355275233,20,0.001,false,saga,9,pca-ridge


In [41]:
pca_ridge_run.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser)

display_pdf(est_grid_results_pdf(pca_ridge_run,
                                 est_tag='pca-ridge-ironore'))

mean_test_score,pca__n_components,rdg__alpha,rdg__normalize,rdg__solver,rank_test_score,est_tag
-1.059965796482985,30,1000.0,true,saga,18,pca-ridge-ironore
-1.059917846009841,20,1000.0,true,saga,17,pca-ridge-ironore
-1.059561453204129,10,1000.0,true,saga,16,pca-ridge-ironore
0.41550871077184864,30,1.0,true,saga,15,pca-ridge-ironore
0.42555084437326063,20,1.0,true,saga,14,pca-ridge-ironore
0.4919238247528998,10,0.001,true,saga,13,pca-ridge-ironore
0.5009833754403592,10,1.0,true,saga,12,pca-ridge-ironore
0.5146670219141831,30,1.0,false,saga,11,pca-ridge-ironore
0.5147309660523389,10,1.0,false,saga,10,pca-ridge-ironore
0.5148976760743146,10,0.001,false,saga,9,pca-ridge-ironore


The best `mean_test_score` with the hyperparameters for the models are:<table>
    <tr>
        <td>feature </td><td>target</td><td>`normalize`</td><td>`alpha`</td><td>`solver`</td><td>`pca_n_components`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>`False`</td><td>1</td><td>`saga`</td><td>20</td><td>50.77%</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>`True`</td><td>0.001</td><td>`saga`</td><td>30</td><td>55.62%</td>
    </tr>
    
</table>

- The hyperparameter which produced the most consistent test score is the `rdg_solver='saga'` followed by `alpha=0.001` with scores above 50%. 
- When `normalize` is `true`, and `alpha` is 0.001 with solver as `saga` with a dimensionality of 30 (`pca_n_component=30`), it gives the best R square value of 55.62%.

### MODEL 7: Decision Tree

Use Decision tree regressor with grid search to find out the best tree depth and maximum leaf nodes. The estimator pipeline used in the gridsearch is `decision_tree_regressor`, and the parameters are as below:
- tree depth (`max_depth`). The range is from 1 to 10. The deeper the tree, the more complex the decision rules and the fitter the model.
- `max_leaf_nodes`. It represents the max leaf nodes that the tree can grow. Here it is determined by the list of numbers [5, 10, 15, 20, 25]. The tree will stop growing once the maximum number is reached.

In [45]:
dtr_run = \
GridSearchCV(sc,
             estimator=get_DecisionTree_pipeline(),
             param_grid={'dtr__max_depth'        : [1,2,3,4,5,6,7,8,9,10],
                         'dtr__max_leaf_nodes': [5, 10, 15, 20, 25]
                        },
             cv=TimeSeriesSplit(n_splits=5),
             scoring=make_scorer(r2_score),
             return_train_score=False,
             n_jobs=-1 
            ) 
dtr_run \
  .fit(trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser)
display_pdf(est_grid_results_pdf(dtr_run,
                                 est_tag='dtr'))


mean_test_score,dtr__max_depth,dtr__max_leaf_nodes,rank_test_score,est_tag
0.2194660207212406,5,20,50,dtr
0.2307677969847591,8,25,49,dtr
0.2308458518799672,10,20,48,dtr
0.2336769211662965,7,20,47,dtr
0.23452739355336447,8,20,46,dtr
0.2587801238949398,4,25,45,dtr
0.26015563872930453,9,20,44,dtr
0.26182580320639726,9,25,43,dtr
0.2644127266655444,6,20,42,dtr
0.29982185770169645,4,20,41,dtr


In [46]:
dtr_run.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser)

display_pdf(est_grid_results_pdf(dtr_run,
                                 est_tag='dtr-ironore'))

mean_test_score,dtr__max_depth,dtr__max_leaf_nodes,rank_test_score,est_tag
0.18203365551903658,7,25,50,dtr-ironore
0.19269687787234963,9,25,49,dtr-ironore
0.21285896616335284,8,25,48,dtr-ironore
0.22849194472556017,8,20,47,dtr-ironore
0.2457448375243338,6,25,46,dtr-ironore
0.25389340941208005,5,25,45,dtr-ironore
0.26618037738132383,6,20,44,dtr-ironore
0.2758673816509936,3,20,43,dtr-ironore
0.27735528832836465,4,20,42,dtr-ironore
0.28407982195354325,10,20,41,dtr-ironore


The test results from the gridsearch with the `decision_tree_regressor` pipeline are:<table>
    <tr>
        <td>feature </td><td>target</td><td>`max_depth`</td><td>`max_leaf_nodes`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>10</td><td>10</td><td>53.93%</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>8</td><td>10</td><td>49.69%</td>
    </tr>
    
</table>

- From the GridseachCV results, it can be clearly seen that none of the hyperparameters are consistent in producing the better scores.
- The best result of an R-square value of 53.93% is produced with the hyperparameters `max_depth` is 10 and `max_leaf_nodes` as 10 with the training dataset produced using the `TfidfVectorizer` method on the `bci_ironore_pdf` dataset.

### MODEL 8: Random Forest

A random forest is simply a collection of decision trees whose results are aggregated into one final result. Their ability to limit overfitting without substantially increasing error due to bias is why they are such powerful models. One way Random Forests reduce variance is by training on different samples of the data.

Use Random Forest regressor with grid search to find out the best tree numbers and leaf nodes. Pipeline is random forest regressor, param_grid includes:
- `n_estimators`: The list of number of trees in forest. Here it is determined by the list of numbers [5,10,20]. 
- `max leaf nodes`: It represents the maximum leaf nodes that the trees can grow. Here it is determined by the list of numbers [5, 10, 15, 20, 25]. The trees will stop growing once the maximum number is reached

The the cross-validation scores for all the models with different combinations of hyperparameters are then averaged out to get the `mean_test_score` and ranked accordingly.

In [50]:
rf_run = \
GridSearchCV(sc,
             estimator=get_RandomForest_pipeline(),
             param_grid={'rf__n_estimators'  : [5, 10, 20],
                         'rf__max_leaf_nodes': [5, 10, 15, 20, 25]
                        },
             cv=TimeSeriesSplit(n_splits=5),
             scoring=make_scorer(r2_score),
             return_train_score=False,
             n_jobs=-1 
            ) 
rf_run \
  .fit(trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser)
display_pdf(est_grid_results_pdf(rf_run,
                                 est_tag='rf'))

mean_test_score,rf__max_leaf_nodes,rf__n_estimators,rank_test_score,est_tag
0.4503485714552817,20,5,15,rf
0.453380895328962,15,5,14,rf
0.4652366039166557,20,10,13,rf
0.46781684168940796,25,5,12,rf
0.4936125763155318,25,20,11,rf
0.4936986297174746,15,10,10,rf
0.4976692186645073,25,10,9,rf
0.5110166952838263,5,5,8,rf
0.5113271513305874,5,20,7,rf
0.5180802183376848,20,20,6,rf


In [51]:
rf_run.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser)

display_pdf(est_grid_results_pdf(rf_run,
                                 est_tag='rf-ironore'))

mean_test_score,rf__max_leaf_nodes,rf__n_estimators,rank_test_score,est_tag
0.431205534334401,5,5,15,rf-ironore
0.4541517420917562,20,10,14,rf-ironore
0.4572879991754535,25,5,13,rf-ironore
0.4970529845685014,20,20,12,rf-ironore
0.506471657109739,25,20,11,rf-ironore
0.5230171786372636,25,10,10,rf-ironore
0.5362305835501161,15,20,9,rf-ironore
0.5366568947837363,5,10,8,rf-ironore
0.5403992994592566,15,10,7,rf-ironore
0.5426649735462066,15,5,6,rf-ironore


The following test results we can get the best hyperparameters for the Random Forest regression model:<table>
    <tr>
        <td>feature </td><td>target</td><td>`n_estimators`</td><td>`max_leaf_nodes`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>20</td><td>10</td><td>58.76%</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>5</td><td>10</td><td>61.46%</td>
    </tr>
    
</table>

- As seen from the above test scores, random forest produces improved `mean_test_scores` than the Decision Tree Model. The most consistent hyperparameter with better test score is `max_leaf_nodes=10`.
- And the best score for the model is when `n_estimators` is 5, and `max_leaf_nodes` is 10, it gives the best mean R square value of 61.46%.

## MODEL EVALUATION AND SELECTION

Using GridsearchCV all the different regression models were investigated with different hyperparameters and tuned to get the best scores for the model. Some of the models improved their r-square values after tuning with different hyperparameters while some of the scores were reduced. 
For example the estimator `ElasticLasso` with the default parameters of `alpha=1`  and `normalize=True` had the highest `mean_test_score` with and R-square value of 72.21% when compared to other models with default values, whereas after the hyperparameter during using GridsearchCV, this reduced to 60.79%.

Similarly the ElasticNet model which had a negative mean test score improved after hyperparameter tuning and gave the best score of 69.18%. 

After testing with different hyperparameter values, here's the score rank of all 8 models with the best outcome:

- Model 1: Elastic Net with PCA: 69.18%
- Model 2: Lasso with PCA: 63.43%
- Model 3: Random Forest: 61.46%
- Model 4: Elastic Net: 60.79%
- Model 5: Lasso Regression: 60.67% 
- Model 6: Ridge Regression: 60.24%
- Model 7: Decision Tree: 53.93%
- Model 8: Ridge with PCA: 52.62%

Although PCA has several advantages, but the main drawback of PCA is that the decision about how many principal components to keep does not depend on the response variable. Consequently, some of the variables that is selected might not be strong predictors of the response, and some of the components that is dropped might be excellent predictors. It does not consider the response variable when deciding which principal components to drop. The decision to drop components is based only on the magnitude of the variance of the components.

Also it makes the independent variables selected less interpretable. Hence as the final models for prediction, the Random Forest model is also selected to compare the prediction score and their accuracies with the ElasticNet model.

From the study on the gridsearch scores for various regression models, the best model to predict the `bci_5tc` price is **Elastic Net with PCA** using the features from the `bci_pdf` and additional features extracted from the `coal_pdf` using `CountVectorizer` method. This model combines the penalties of both ridge regression and lasso to take into account of overfitting and to get the best of both worlds.

The best hyperparameters for the two models to be compared and get the test score for predicting are:

  - **Model 1: ElasticNet with PCA**<table>
    <tr>
        <td>Feature</td><td>Target</td><td>**`Normalize`**</td><td>**`Alpha`**</td><td>**`Pca_n_components`**</td><td>R Square</td>
    </tr>
    <tr>
        <td>`trn_coal_cnt_fea_pdf`</td><td>`trn_coal_cnt_tgt_ser`</td><td>`true`</td><td>0.001</td><td>100</td><td>69.18%</td>
    </tr>
    
</table>

  - **Model 2: Random Forest**<table>
    <tr>
        <td>Feature</td><td>Target</td><td>`n_estimators`</td><td>`max_leaf_nodes`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>`trn_ore_tfidf_fea_pdf`</td><td>`trn_ore_tfidf_tgt_ser`</td><td>5</td><td>10</td><td>61.46%</td>
    </tr>
    
</table>

**Model 1: ElasticNet with PCA**
The function for the PCA ElasticNet pipeline model with the hyperparameters selected from the GridsearchCV results is defined and this model is tested on the test dataset.

In [57]:
%python 
def tst_pca_elasticnet_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.linear_model import LinearRegression, Ridge
  return Pipeline(steps=[
    ('pca', FeatureSelectionPCA(n_components=100)),
    ('ela', EstimatorElasticNet(normalize=True, alpha=0.001))
  ])

In [58]:
tst_pca_elasticnet_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[120]: 0.5780285370376351

The r-square value for the prediction is 57.8% which is a bit lower than the r-square value of 69% on the training dataset. 

The sklearn.metrics module implements several loss, score, and utility functions to measure regression performance. Metrics are used to evaluate a model by comparing the actual values with the predicted values produced by the model. Some of these metrics used to evaluate the regression model are: `mean_squared_error`, `mean_absolute_error`, `explained_variance_score`, `mean_squared_log_error`, `median_absolute_error` and `r2_score`. Inorder to compare the predicted value to the observed response value, the predicted `bci_5tc` values are calculated using `.predict()` to the fitted model.

In [60]:
predicted = \
tst_pca_elasticnet_pipeline() \
.fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
.predict(tst_coal_cnt_fea_pdf)

observed = np.array(tst_coal_cnt_tgt_ser)


In [61]:
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score

print('Explained Variance Score : ' + str(explained_variance_score(observed,predicted)))
print('Mean Absolute Error      : ' + str(mean_absolute_error(observed,predicted)))
print('Mean Squared Error       : ' + str(mean_squared_error(observed,predicted)))
print('Mean Squared Log Error   : ' + str(mean_squared_log_error(observed,predicted)))
print('Median Absolute Error    : ' + str(median_absolute_error(observed,predicted)))
print('R Squared (R2)           : ' + str(r2_score(observed,predicted)))

Explained Variance Score : 0.7615190914515925
Mean Absolute Error : 2821.2498234059735
Mean Squared Error : 11684833.82181007
Mean Squared Log Error : 0.04215121039563557
Median Absolute Error : 2459.803943162038
R Squared (R2) : 0.5784239635514794

**Model 2: Random Forest**
The random forest model is defined with the hyperparameters selected from the Gridsearch results, `n_estimators=5` and `max_leaf_nodes=10`. This model is then trained using the `bci_ironore_tfidf` training dataset and then tested with the testing dataset. The scores and metrics are calculated below. The R-square value for prediction is 66.05% which is better than the training score of 61%.

In [63]:
from sklearn.tree     import DecisionTreeRegressor
rf = RandomForestRegressor(n_estimators=5, max_leaf_nodes=10)

rf.fit(trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
.score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)


Out[136]: 0.6605627254541642

In [64]:
rf_predicted = rf.fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser).predict(tst_ore_tfidf_fea_pdf)
rf_observed = np.array(tst_ore_tfidf_tgt_ser)

Measure the regression performance for the model and compare the two models.

In [66]:
print('Explained Variance Score : ' + str(explained_variance_score(rf_observed,rf_predicted)))
print('Mean Absolute Error      : ' + str(mean_absolute_error(rf_observed,rf_predicted)))
print('Mean Squared Error       : ' + str(mean_squared_error(rf_observed,rf_predicted)))
print('Mean Squared Log Error   : ' + str(mean_squared_log_error(rf_observed,rf_predicted)))
print('Median Absolute Error    : ' + str(median_absolute_error(rf_observed,rf_predicted)))
print('R Squared (R2)           : ' + str(r2_score(rf_observed,rf_predicted)))

Explained Variance Score : 0.6078652538948579
Mean Absolute Error : 2532.432016352566
Mean Squared Error : 11055056.546956075
Mean Squared Log Error : 0.04087421194505001
Median Absolute Error : 2002.3375360184473
R Squared (R2) : 0.6012848295081206

When the metrics scores for the two model performance are compared, it is observed that all the error scores are less for Random Forest compared to that of the ElasticNet model. As the R-square value for the Random Forest model is better for the prediction on test sets with a score of 66%, it can be concluded to be the better fit model to predict the `bci_5tc` price. 

Also for random forest models, important features for the prediction model can also be calculated with their importance value. It is calculated using the `feature_importance_` method in sklearn. 
Feature importance is calculated as the decrease in node impurity weighted by the probability of reaching that node. The node probability can be calculated by the number of samples that reach the node, divided by the total number of samples. The higher the value the more important the feature.
From the results below, it is noticed that the variables `bci_`, `c5_` are the most important features to give an accurate prediction with the model.

In [68]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = trn_ore_tfidf_fea_pdf.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

Out[139]: 
 importance
bci_lag3 0.660459
c5_lag3 0.302947
p4_03_lag3 0.014081
dayofyear 0.004419
p3a_03_lag3 0.003285
year 0.003127
shfe_cu3_lag3 0.002969
shfe_al3_lag3 0.002231
cme_s2_lag3 0.001963
p3a_iv_lag3 0.001704
ice_tib3_lag3 0.001460
tfidf_content_added 0.001355
tfidf_content_realizing 0.000000
tfidf_content_reaping 0.000000
tfidf_content_reapply 0.000000
tfidf_content_rear 0.000000
tfidf_content_rearing 0.000000
tfidf_content_reason 0.000000
tfidf_content_reasonable 0.000000
tfidf_content_reasonableâ 0.000000
tfidf_content_reasonably 0.000000
tfidf_content_reasoning 0.000000
tfidf_content_reasons 0.000000
tfidf_content_reasonâ 0.000000
tfidf_content_reassembled 0.000000
tfidf_content_reassert 0.000000
tfidf_content_reassess 0.000000
tfidf_content_reassessment 0.000000
tfidf_content_reassign 0.000000
tfidf_content_reappeared 0.000000
... ...
tfidf_content_eremin 0.000000
tfidf_content_erg 0.000000
tfidf_content_ergo 0.000000
tfidf_content_ergoâ 0.000000
tfidf_content_eric 0.000000
tfidf_content_erica 0.000000
tfidf_content_erick 0.000000
tfidf_content_ericsson 0.000000
tfidf_content_ericâ 0.000000
tfidf_content_erik 0.000000
tfidf_content_eraâ 0.000000
tfidf_content_erase 0.000000
tfidf_content_equipped 0.000000
tfidf_content_eradicated 0.000000
tfidf_content_equitable 0.000000
tfidf_content_equitably 0.000000
tfidf_content_equities 0.000000
tfidf_content_equitise 0.000000
tfidf_content_equity 0.000000
tfidf_content_equity333 0.000000
tfidf_content_equity6 0.000000
tfidf_content_equity7 0.000000
tfidf_content_equityâ 0.000000
tfidf_content_equivalent 0.000000
tfidf_content_equivalents 0.000000
tfidf_content_equivalentâ 0.000000
tfidf_content_er 0.000000
tfidf_content_era 0.000000
tfidf_content_eradicate 0.000000
tfidf_title_zuckerberg 0.000000

[39616 rows x 1 columns]

From the above findings the Random Forest Model is selected as the best model for the prediction of the target variable `bci_5tc`. The features used for the model are the lagged version of all features from the time series dataset `bci_pdf` and the extracted features from the mining text dataset `iron_ore_pdf` using `TfidfVectorizer` method.

##SUMMARY: 
In this report different machine learning classes and feature selection techniques are used. Several pipelines with different estimators and wrapper classes were used to design the regression models which were then investigated to get the best hyperparameters using GridsearchCV. The `mean_test_scores` for each model with different hyperparameters were compared for each model and ranked accordingly in the section **Hyperparameter Tuning**. Ultimately the best two models **ElasticNet with PCA and Random Forest models** were selected as the final models to predict using the test datasets and compare their results with the observed response variable in model evaluation.

Out of the two models with their tuned hyperparameters, Random forest has the best test score with lower error metrics and interpretable predictor variables which can be used to explain the model better.